In [15]:
from treelib import Node, Tree

#create tree object from an annotation file
#Sample Annotation:
#
#
# What is the way things are?
# >###Q_1###
# >>###Answer_1’###
# >>###Question_2###
# >>>###Answer_2’###
# >>>###Answer_2’’###
# >>###Question_3###
# >>>###Answer_3###
# >>>###Question_4###
# >>>>###Answer_4###
# >>###Answer_1’’###
#
def createTreeFromFile(filename):
    tree = Tree()
    with open (filename, "r") as myfile:
        data=myfile.readlines()
    level_dict = dict()
    node_no=0
    tree.create_node(getText(data[0]), node_no)
    level_dict[0] = node_no
    for node in data[1:]:
        node_no = node_no+1
        level = getLevel(node)
        
        text = getText(node)
        node_type = getType(node)
        tree.create_node(text,node_no,parent=level_dict[level-1])
        level_dict[level] = node_no

    tree.show(key=lambda x: x.identifier)

    return tree

In [16]:
import re
def getText(string):
    p = re.compile(">*(.*)\n")
    return p.match(string).group(1)

def getLevel(string):
    p = re.compile(">*")
    return p.match(string).span()[1]-p.match(string).span()[0]

def getType(string):
    p = re.compile(">*(A|Q*)")
    return p.match(string).group(1)

In [23]:
createTreeFromFile("Snowden/nguyen-snowden.txt")

What is the way things are?
├── How are you? How does that come about?
│   └── Mr. Snowden, did you sleep well the last couple of nights?
└── Why am I asking this?
    ├── Because, I was reading that you asked for a kind of police protection.
    ├── Why did you ask for police protection?
    └── Are there any threats?
        ├── Are there any threats?
        └── There are significant threats but I sleep very well.
            ├── What (would be an example of such a threat) kind of threat?
            ├── There was an article that came out in an online outlet called BuzzFeed
            │   ├── What kind of article?
            │   └── where they interviewed officials from the Pentagon, from the National Security Agency
            │       ├── How did they (get them to talk) interview them?
            │       ├── and they gave them anonymity to be able to say what they wanted,
            │       ├── So what did they say?
            │       ├── and what they told the reporter was t